 # 개발 환경 OS

In [ ]:
# !nvidia-smi

Thu Mar 27 08:32:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# !head /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0


In [ ]:
# !head -n 3 /proc/meminfo

MemTotal:       13290460 kB
MemFree:         6806628 kB
MemAvailable:   11764300 kB


In [ ]:
# import sys
# print(f"Python version: {sys.version}")

Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]


# Library

In [ ]:
!pip install torch

In [ ]:
%%capture
!pip install langchain-community langchain-text-splitters langchain langchain-huggingface rank_bm25 transformers torch faiss-cpu pymupdf datasets

In [ ]:
import os
import torch
import random
import pandas as pd
import langchain
import langchain_community
import langchain_huggingface
import transformers
import pkg_resources
import sentence_transformers

from tqdm import tqdm

from transformers import  BartForConditionalGeneration, PreTrainedTokenizerFast
from sentence_transformers import SentenceTransformer

from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.retrievers import EnsembleRetriever, ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

from langchain_huggingface import HuggingFaceEmbeddings

from langchain.prompts import PromptTemplate

In [ ]:
# print(f"torch version: {torch.__version__}")
# print(f"pandas version: {pd.__version__}")
# print(f"tqdm version: {pkg_resources.get_distribution('tqdm').version}")
# print(f"transformers version: {transformers.__version__}")
# print(f"sentence_transformers version: {sentence_transformers.__version__}")
# print(f"langchain version: {langchain.__version__}")
# print(f"langchain_community version: {langchain_community.__version__}")
# print(f"langchain_huggingface version: {pkg_resources.get_distribution('langchain_huggingface').version}")

torch version: 2.6.0+cu124
pandas version: 2.2.2
tqdm version: 4.67.1
transformers version: 4.50.0
sentence_transformers version: 3.4.1
langchain version: 0.3.21
langchain_community version: 0.3.20
langchain_huggingface version: 0.1.2


# 경로 설정

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# path = '/content/drive/MyDrive/# 협동 업무/한솔데코 데이콘/A3/' # Colab에서 실행 시
path = '/' # Local 환경에서 실행 시
db_path = path + 'DB/'
vectorstore_path = path + 'VectorStore/'
model_path = path + 'Model/'
score_path = path + 'TestScore/'
submission_path = path + 'Submission/'

# PDF File 경로
pdf_path = path + '건설안전지침/'
file_list = os.listdir(pdf_path)

# 데이터 로드 및 전처리

In [ ]:
test = pd.read_csv(db_path + 'test_preprocessing.csv')

# 데이터 전처리
test['공사종류(대분류)'] = test['공사종류'].str.split(' / ').str[0]
test['공사종류(중분류)'] = test['공사종류'].str.split(' / ').str[1]
test['공종(대분류)'] = test['공종'].str.split(' > ').str[0]
test['공종(중분류)'] = test['공종'].str.split(' > ').str[1]
test['사고객체(대분류)'] = test['사고객체'].str.split(' > ').str[0]
test['사고객체(중분류)'] = test['사고객체'].str.split(' > ').str[1]

In [ ]:
# test data 전처리
data_test = test.apply(
    lambda row: {
        "process": row["작업프로세스"],
        "construct_type": row["공종(중분류)"],
        "object_type": row["사고객체(중분류)"],
        "reason": row['사고원인'], # 사고원인 추가
        "situation": (
            f"'{row['공사종류(대분류)']}' 공사 중 '{row['공종(중분류)']}' 작업 중 '{row['사고원인']}'으로 인해 사고가 발생하였습니다."),
    },
    axis=1
)

# DataFrame으로 변환
data_test = pd.DataFrame(list(data_test))

# Test 입력 데이터 설정
query = data_test["situation"].tolist() # 질문 데이터 리스트로 변환

# # pdf 뽑아낼 construct_type_query 설정
construct_type_query = data_test["construct_type"].tolist()  # "공종(중분류)"을 construct_type_query로 사용

In [ ]:
data_test.head()

,process,construct_type,object_type,reason,situation
0,타설작업,철근콘크리트공사,콘크리트펌프,"펌프카 아웃트리거 바닥 고임목을 3단으로 보강 했음에도, 지반 침하(아웃트리거 우측...",'건축' 공사 중 '철근콘크리트공사' 작업 중 '펌프카 아웃트리거 바닥 고임목을 3...
1,절단작업,수장공사,공구류,작업자의 불안전한 행동(숫돌 측면 사용) 및 보안면 미 착용,'건축' 공사 중 '수장공사' 작업 중 '작업자의 불안전한 행동(숫돌 측면 사용) ...
2,이동,미장공사,기타,작업자가 작업을 위해 이동 중 전방을 주시하지 않아 발을 헛디뎌 계단에서 굴러 넘어짐,'건축' 공사 중 '미장공사' 작업 중 '작업자가 작업을 위해 이동 중 전방을 주시...
3,기타,조적공사,자재,작업 발판 위 벽돌 잔재를 밟고 넘어짐,'건축' 공사 중 '조적공사' 작업 중 '작업 발판 위 벽돌 잔재를 밟고 넘어짐'으...
4,해체작업,교량공사,기타,점심식사를 위한 이동시 작업자 부주의로 인한 추락사고 발생,'토목' 공사 중 '교량공사' 작업 중 '점심식사를 위한 이동시 작업자 부주의로 인...


# 함수 선언

## 1) Chunking 함수

In [ ]:
# 1. Content Chunking 함수 => 의미 기반
def chunking():
    documents = []
    paths = []
    total_columns = 0

    # 텍스트 분할기
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=size, chunk_overlap=overlap)

    # Chunking
    for idx, file in enumerate(file_list):
        # 1. 텍스트 파일을 load -> List[Document] 형태로 변환
        loader = PyMuPDFLoader(pdf_path + file)

        # 2. 문서 분할
        chunks = loader.load_and_split(text_splitter)

        # 3. 각 청크를 벡터화하고 삽입
        for chunk in chunks:
            documents.append(chunk.page_content)
            paths.append(file)

        print("="*50)
        print(f"{idx+1} 파일 명: {file}, 분할 개수: {len(chunks)}")

        # 4. 전체 컬럼 수
        # 전체 컬럼 수 세기
        total_columns += len(chunks)
        # 전체 컬럼 수 출력
        if idx == len(file_list)-1:
            print(f"DB 총 Column 수 : {total_columns}")

    return documents, paths

# 2.sparse 기반 chunking => 키워드 기반
def sparse_chunking(file: str):
    documents = []
    total_columns = 0

    # 텍스트 분할기
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=size, chunk_overlap=overlap)

    # Chunking
    # 1. 텍스트 파일을 load -> List[Document] 형태로 변환
    loader = PyMuPDFLoader(pdf_path + file)

    # 2. 문서 분할
    chunks = loader.load_and_split(text_splitter)

    # 3. 각 청크를 벡터화하고 삽입
    for chunk in chunks:
        documents.append(chunk.page_content)

    # 4. 전체 컬럼 수
    # 전체 컬럼 수 세기
    total_columns += len(chunks)

    print(f"DB 총 Column 수 : {total_columns}")

    return documents


## 2) VectorStore 함수

In [ ]:
# 3. Title_DenseVectorDB 생성 및 저장
def title_vector_store_save(folder_name: str, file_list: list):
    # PDF 제목 리스트
    pdf_titles = [file.replace('.pdf', '') for file in file_list]

    # 임베딩 모델
    embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)
    print("DB 생성 중...")
    db = FAISS.from_texts(pdf_titles,
                        embedding=embedding_model,
                        # docstore=InMemoryDocstore(),
                        metadatas=[{"path": p} for p in file_list])
    print("DB 생성 완료")

    print("DB 저장 중...")
    db.save_local(folder_name)
    print("DB 저장 완료")

# 4. Content_DenseVectorDB 생성 및 저장
def content_vector_store_save(folder_name: str):
    # 임베딩 모델
    embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)

    documents, paths = chunking()
    print("DB 생성 중...")
    db = FAISS.from_texts(documents,
                        embedding=embedding_model,
                        # docstore=InMemoryDocstore(),
                        metadatas=[{"path": p} for p in paths])
    print("DB 생성 완료")

    print("DB 저장 중...")
    db.save_local(folder_name)
    print("DB 저장 완료")

# #VectorDB에 PDF Chunk화 하여 저장
# title_vector_store_save('Title_DB',file_list)
# content_vector_store_save('Content_DB')

# 5. 저장된 벡터 스토어 로드
def vector_store_load(folder_name: str):
    """
    FAISS 벡터 DB 로드 함수
    """
    faiss_path = vectorstore_path + f"{folder_name}"  # 절대경로 사용

    # FAISS 인덱스 파일이 존재하는지 확인
    if not os.path.exists(f"{faiss_path}/index.faiss"):
        raise RuntimeError(f"FAISS 인덱스 파일을 찾을 수 없습니다: {faiss_path}/index.faiss")

    # FAISS 벡터 DB 로드
    embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-sts")
    vector_store = FAISS.load_local(faiss_path, embedding_model, allow_dangerous_deserialization=True)

    print(f"FAISS 벡터 DB 로드 성공: {faiss_path}")
    return vector_store

## 3) Combine Chunked Text 함수

In [ ]:
# 4. 검색한 청크들을 하나의 청크로 합치는 함수.
def format_docs(docs):
    """
    검색한 청크들을 하나의 청크로 합치는 함수.

    Args:
        docs (list): Faiss로 부터 검색한 내용을 담은 리스트

    Returns:
        list: 하나의 청크로 합친 리스트
    """
    return "\n".join(
        [
            f"{doc.page_content}"
            for doc in docs
        ]
    )

# Model 로드

In [ ]:
model_name = 'kobart7'

# 저장된 모델 경로
load_model_path = model_path + model_name

# 모델 & 토크나이저 로드
model = BartForConditionalGeneration.from_pretrained(load_model_path, device_map="auto", num_labels=2)
tokenizer = PreTrainedTokenizerFast.from_pretrained(load_model_path)
print("모델과 토크나이저가 정상적으로 로드되었습니다!")

print("pad_token:", tokenizer.pad_token)
print("pad_token_id:", tokenizer.pad_token_id)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


모델과 토크나이저가 정상적으로 로드되었습니다!
pad_token: </s>
pad_token_id: 1


# Retriever

## VectorStore 로드


In [ ]:
# chunk size 조정
size = 300
overlap = 50

# Hybrid Retriever 함수로 구성 (3개의 pdf)
title_db = vector_store_load("Title_DB")
content_db = vector_store_load(f"Content_DB_{size}_{overlap}")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.00k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FAISS 벡터 DB 로드 성공: /content/drive/MyDrive/# 협동 업무/한솔데코 데이콘/A3/VectorStore/Title_DB
FAISS 벡터 DB 로드 성공: /content/drive/MyDrive/# 협동 업무/한솔데코 데이콘/A3/VectorStore/Content_DB_300_50


In [ ]:
# re-rank retriever 함수로 구성 (3개의 pdf)
def rerank_retriever(construct_type_query: str, query: str, k: int = 2):
    """
    Fine-Tuned LLaMA + Hybrid Retriever 기반 RAG 실행 함수
    construct_type_query: PDF 검색에 사용할 '공종(중분류)' 정보
    query: 실제 검색에 사용할 '사고상황'
    k: 검색할 PDF 개수
    """
    print(f"🔍 Hybrid Retriever 시작 → construct_type_query: {construct_type_query}, query: {query}")

    # 1. 저장된 FAISS 벡터 DB 로드
    # embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)
    title_db = vector_store_load("Title_DB")

    # 2. FAISS 벡터 겁색 수행 -> '공종(중분류)'으로 가장 유사한 PDF 찾기
    result = title_db.similarity_search(construct_type_query, k=3)  # 가장 유사한 3개 찾기

    # 검색된 PDF 파일 목록
    pdfs = [doc.metadata["path"] for doc in result]
    print(f" 선택된 PDF: {pdfs}")

    # 3. Content DB 로드 (선택된 PDF에서 검색)
    content_db = vector_store_load(f"Content_DB_{size}_{overlap}")

    # FAISS 리트리버 생성 (Dense Retriever)
    dense_retriever = content_db.as_retriever(
        search_type="similarity",
        search_kwargs={"filter": {"path": {"$in": pdfs}}, "k": k}  # 선택된 PDF에서 검색
    )

    # 4. BM25를 위한 Sparse 검색 수행
    all_documents = []
    for pdf in pdfs:
        # BM25 검색용 문서 객체 생성
        documents = [Document(page_content=doc) for doc in sparse_chunking(pdf)]
        all_documents.extend(documents)

    sparse_retriever = BM25Retriever.from_documents(all_documents)

    # 5. Hybrid Retriever 생성 (FAISS + BM25 결합)
    ensemble_retriever = EnsembleRetriever(
        retrievers=[sparse_retriever, dense_retriever],
        weights=[0.7, 0.3]  # 가중치 설정
    )

    # Re-Rank 모델 설정
    reranker_model_name = "BAAI/bge-reranker-v2-m3"  # Hugging Face Re-Ranker 모델

    rerank_model = HuggingFaceCrossEncoder(model_name=reranker_model_name)
    compressor = CrossEncoderReranker(model=rerank_model, top_n=k) # 가져오고자 하는 청크의 갯수를 top_n에 기입

    # re-ranker 적용
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=ensemble_retriever
    )

    return compression_retriever

def hybrid_retriever(construct_type_query: str, query: str, k: int = 2):
    """
    Fine-Tuned LLaMA + Hybrid Retriever 기반 RAG 실행 함수
    construct_type_query: PDF 검색에 사용할 '공종(중분류)' 정보
    query: 실제 검색에 사용할 '사고상황'
    k: 검색할 PDF 개수
    """
    # 1. FAISS 벡터 겁색 수행 -> '공종(중분류)'으로 가장 유사한 PDF 찾기
    result = title_db.similarity_search(construct_type_query, k=3)  # 가장 유사한 3개 찾기

    # 검색된 PDF 파일 목록
    pdfs = [doc.metadata["path"] for doc in result]

    # 2. Content DB 로드 (선택된 PDF에서 검색)

    # FAISS 리트리버 생성 (Dense Retriever)
    dense_retriever = content_db.as_retriever(
        search_type="similarity",
        search_kwargs={"filter": {"path": {"$in": pdfs}}, "k": k}  # 선택된 PDF에서 검색
    )

    # 3. BM25를 위한 Sparse 검색 수행
    all_documents = []
    for pdf in pdfs:
        # BM25 검색용 문서 객체 생성
        documents = [Document(page_content=doc) for doc in sparse_chunking(pdf)]
        all_documents.extend(documents)

    sparse_retriever = BM25Retriever.from_documents(all_documents)

    # 4. Hybrid Retriever 생성 (FAISS + BM25 결합)
    ensemble_retriever = EnsembleRetriever(
        retrievers=[sparse_retriever, dense_retriever],
        weights=[0.2, 0.8]  # 가중치 설정
    )

    return ensemble_retriever

# ensemble retriever에서 query관련 청크들 뽑아오기
def retrieved_chunk(construct_type_query: str, query: str, k: int = 3):
    """
    Hybrid Retriever를 사용하여 query에 대한 관련 청크 검색
    """
    ensemble_retriever = hybrid_retriever(construct_type_query, query, k)

    # 검색된 관련 청크 리스트 반환
    retrieved_chunks = ensemble_retriever.invoke(query)

    # 검색된 청크를 하나의 텍스트로 변환
    retrieved_text = format_docs(retrieved_chunks)

    return retrieved_text

# Prompt

In [ ]:
# LangChain Prompt Template 생성(파인튜닝한 템플릿 유지 + 검색된 청크 활용)
prompt_template = PromptTemplate(
    input_variables=["process","construct_type", "object_type", "context", "situation"],
    template=(
'''
제공된 data는 건설 공사현장에서 발생한 사고 상황입니다. 주어진 situation을 분석하고 context를 참고해 재발방지 대책을 포함한 대응책을 response에 작성하세요.
### context:
{context}

### process:
{process}

### construct_type:
{construct_type}

### object_type:
{object_type}

### situation:
{situation}

※ Example response:
- 작업자 안전교육 및 재발 방지 대책 수립과 작업 전 안전교육 철저 및 관리자 추가 배치를 통한 동종 사고 예방.
- 근로자 보행 통로 구간 안전표지판 설치와 특별안전교육 실시, 일일 작업 투입 전 상시 교육, 관리 대상 선정 등을 통한 이동 구간 보행 안전 확보와 작업자 안전 교육 지시.

### response:
''')
)

# 추론

In [ ]:
# generate_text 함수
def generate_answer(ts, prompt, model, tokenizer, max_new_tokens=65):
    """
    주어진 프롬프트를 기반으로 모델이 재발 방지 대책 및 향후 조치 계획을 생성하는 함수
    """

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    tokens = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1024
    )


    process = ts['process']
    construct_type = ts['construct_type']
    object_type = ts['object_type']
    situation = ts['situation']
    reason = ts['reason']


    # 입력 문장 토큰화 및 GPU로 이동
    input_ids = tokens.input_ids.to(device)
    # print('prompt:',tokenizer.decode(input_ids[0], skip_special_tokens=True))
    attention_mask = tokens.attention_mask.to(device)

    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_k=40,
        top_p=0.85,
        repetition_penalty=1.2,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id  # <-- 이 부분이 핵심
    )


    # 생성된 토큰을 다시 문장으로 변환
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # ✅ 후처리: 마지막 마침표(.)까지만 자르기
    if '.' in generated_text:
        last_period_index = generated_text.rfind('.')  # 마지막 마침표 위치
        generated_text = generated_text[:last_period_index + 1].strip()
    else:
        generated_text = generated_text.strip()


    return generated_text


# RAG 실행 (Hybrid Retriever + kobart)
def rag_pipeline(ts, model, tokenizer, k=2):
    """
    RAG 시스템 실행: Hybrid Retriever → Fine-tuned kobart 기반 생성
    """
    # print("🔍 RAG 실행 시작...")

    # Hybrid Retriever 실행하여 관련 문서 청크 검색
    retrieved_text = retrieved_chunk(ts['construct_type'], ts['situation'], k) # 'situation'(사고상황)에서 'reason'(사고원인)으로 변경


    # 프롬프트 포맷팅
    formatted_prompt = prompt_template.format(process=ts['process'],
                                              construct_type=ts['construct_type'],
                                              object_type=ts['object_type'],
                                              context=retrieved_text,
                                              situation=ts['situation'])

    # 문장 생성 함수 호출
    generated_response = generate_answer(ts, formatted_prompt, model, tokenizer)

    return generated_response


## 추론 실험

In [ ]:
# 랜덤하게 10개의 인덱스 선택
random_indices = random.sample(range(len(data_test)), 10)

for i in random_indices:
    ts = data_test.iloc[i]
    process = ts['process']
    construct_type = ts['construct_type']
    object_type = ts['object_type']
    situation = ts['situation']
    generated_text = rag_pipeline(ts, model, tokenizer)
    print(f"사고 상황: {situation}")
    print(f"생성된 문장: {generated_text}")
    print('-' * 100)

DB 총 Column 수 : 44
DB 총 Column 수 : 40
DB 총 Column 수 : 27
사고 상황: '건축' 공사 중 '가설공사' 작업 중 '항타기 SHEAVE 교체 중 작업 중 본인의 좌측 첫번째 손가락을 망치로 타격함'으로 인해 사고가 발생하였습니다.
생성된 문장: 작업 전 안전교육 철저 및 이동 시 올바른 착용 방법 교육을 통한 재발 방지 대책. TBM활동 강화와 교육 실시를 통해 향후 조치 계획으로 안전교육을 철저히 시행하는 방안.
----------------------------------------------------------------------------------------------------
DB 총 Column 수 : 43
DB 총 Column 수 : 27
DB 총 Column 수 : 62
사고 상황: '건축' 공사 중 '철근콘크리트공사' 작업 중 '재해자가 옥탑 난간에 기대고 있는것을 옆에 같이 일하고 있는 형틀공이 확인 하고 이상함을 느껴 재해자를 바닥에 눞혀 CPR실시, 10시 30분경 구급대원 도착하여 CPR 및 AED 병행실시 11시 10분 재해자 119 구급차를 타고 갑을 장유병원 이송'으로 인해 사고가 발생하였습니다.
생성된 문장: 작업자 안전교육 철저에 대한 재발 방지 대책과 향후 조치 계획. 발주처의 감리감독 강화 및 시공사와 시공사간의 의견 조율. 협의 후 이행 여부 결정. 시행. 승인. 승인.
----------------------------------------------------------------------------------------------------
DB 총 Column 수 : 68
DB 총 Column 수 : 28
DB 총 Column 수 : 30
사고 상황: '토목' 공사 중 '관공사' 작업 중 '작업자의 일부 과실(현장에서 인정함)이 있지만 더 많은 인력이 투입되어서 중량이 나가는 콤팩터를 옮기면 사고 예방에 도움이 되었을 것으로 예상됨'으로 인해 사고가 발생하였습니다.
생

# Submission

In [ ]:
# 테스트 실행 및 결과 저장
test_results = []

print("테스트 실행 시작... 총 테스트 샘플 수:", len(data_test))

for idx, row in tqdm(data_test.iterrows(), total=len(data_test), desc="Processing"):
    # RAG 체인 호출 및 결과 생성
    generated_text = rag_pipeline(row, model, tokenizer)  # 개별 행(row) 전달
    test_results.append(generated_text)

print("\n테스트 실행 완료! 총 결과 수:", len(test_results))

테스트 실행 시작... 총 테스트 샘플 수: 964


Processing:   0%|          | 0/964 [00:00<?, ?it/s]

DB 총 Column 수 : 43
DB 총 Column 수 : 27
DB 총 Column 수 : 62


Processing:   0%|          | 1/964 [00:00<09:55,  1.62it/s]

DB 총 Column 수 : 44
DB 총 Column 수 : 49
DB 총 Column 수 : 36


...

DB 총 Column 수 : 42
DB 총 Column 수 : 32
DB 총 Column 수 : 16


Processing: 100%|█████████▉| 963/964 [12:17<00:00,  1.41it/s]

DB 총 Column 수 : 44
DB 총 Column 수 : 49
DB 총 Column 수 : 36


Processing: 100%|██████████| 964/964 [12:18<00:00,  1.31it/s]


테스트 실행 완료! 총 결과 수: 964


In [ ]:
# 2. 문장 임베딩 생성
embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(964, 768)


In [ ]:
# 3. 결과를 CSV 파일로 저장
sample_submission_path = submission_path + 'sample_submission.csv'
submission = pd.read_csv(sample_submission_path, encoding='utf-8-sig')

# 예측 결과 저장
submission.iloc[:, 1] = test_results
submission.iloc[:, 2:] = pred_embeddings  # 임베딩 벡터 저장

# CSV 저장
save_submission_path = submission_path + f'{model_name}_submission.csv'
submission.to_csv(save_submission_path, index=False, encoding='utf-8-sig')

print(f"최종 결과 저장 완료: {save_submission_path}")

최종 결과 저장 완료: /content/drive/MyDrive/# 협동 업무/한솔데코 데이콘/A3/Submission/kobart7_submission.csv
